For my project, I am using climate change precipitation changes (as a proxy for flooding/hurricanes) to predict future housing real estate prices in the continental U.S. While climate change is already established, humans have done a poor job so far adapting to the new changes. The future of our economy is dependent on climate change and its effects on the United States geography, including from hurricanes, flooding, and other extreme weather events and it would benefit people greatly if they were aware of the places best suited to live, where new city hubs are likely to be centered in the future and where forward-thinking businesses will begin focusing on now. To do this, I plan to use either Random Forests or Matrix Factorization to distinguish the role of precipitation in determining the cost of an area and use that along with expected future temperature to predict housing prices in the future. By placing these expected costs on an average precipitation map of the US (by season), individuals and families can gain a better understanding of where the future of their life is. 

In [1]:
#magic getting igor-like graphs
#%matplotlib widget

import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import sqlite3
from sqlite3 import Error
from datetime import datetime
import calendar
from pandas.tseries.offsets import MonthEnd
import time

[] need to match house dates with T_Pr dates
[] plot house prices for T_Pr vs zip code
[] get sum yearly precipitation vs average yearly house price

In [2]:
#Load SQL
def create_connection(path):

    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")
    return connection

create_connection("C:/Users/Aroob Abdelhamid/earth-analytics/housing.sqlite") #create a new database first before running this

Connection to SQLite DB successful


In [3]:
#load city_info first to get station names from ID, and get 'Lon' and 'Lat' and append the loc info to loaded csv with the same filename as ID in city_info
def load_temp_precip_files():
    path = r'C:\Users\Aroob Abdelhamid\earth-analytics\DataIncubator\CMU_TempPrecip_Meas'
    #load city_info first
    city_info = pd.read_csv(path+"\city_info.csv")
    
    all_files = []
    
    #rest of the files   
    T_Pr_files = glob.glob(path + "/US*.csv")#"/US*.csv")
    for filename in T_Pr_files:
        f = pd.read_csv(filename, header=0)
        nm_ext = os.path.basename(filename); nm = os.path.splitext(nm_ext) #get just the filename to search in city_info
        
        #get city_info_data into precip data
        ID_ind = city_info.index[city_info['ID'] == nm[0]]
        Lat = city_info['Lat'][ID_ind[0]]; Lon = city_info['Lon'][ID_ind[0]]
        f['Lat'] = Lat; f['Lon'] = Lon      
        f['Date']= pd.to_datetime(f['Date'], format = '%Y-%m')+MonthEnd(0)
        f['year']= f['Date'].dt.year; f['month']= f['Date'].dt.month; f['day']= f['Date'].dt.day
        f_avg = f.groupby(['year','month'], sort=False, as_index=False).mean().reset_index()
        
        test = f_avg[['year', 'month', 'day']]
        f_avg['Date']= pd.to_datetime(test, format='%Y-%m-%d') 
        
#       put it all together
        all_files.append(f_avg)
    
    T_Pr = pd.concat(all_files, axis=0, ignore_index=True)
    T_Pr = T_Pr.set_index(['Date'])
    return T_Pr,f

#load_temp_precip_files()
T_Pr,f = load_temp_precip_files()

In [4]:
#only keep the dates that zillow has, i.e. after 1996
T_Pr_dates = T_Pr.loc['1996':'2020'].reset_index()

C:/Users/AROOBA~1/AppData/Local/Temp/xpython_8252/94337530.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  T_Pr_dates = T_Pr.loc['1996':'2020'].reset_index()


In [ ]:
plt.scatter(T_Pr['Lon'], T_Pr['Lat'], vmin=0, vmax=0.1 , c= T_Pr['prcp'], cmap='gray')
plt.xlabel('longitude')
plt.grid(axis='both', alpha=0.5)
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Precip (in)', rotation=270)
plt.ylabel('latitude')
plt.title('Map of Measured Precipitation (in)')

As you can see, the precipitation is highest at the coasts and specifically in the SE. As climate change makes hurricanes more intense and changes precipitation patterns, I expect the areas in yellow to be some of the hardest hit, and that is where I expect the housing market to show the most changes. We can inspect that by comparing the housing market prices to these values and seeing if they changed over time.

In [6]:
def load_house_prices_data():
    #load files
    path = r'C:\Users\Aroob Abdelhamid\earth-analytics\DataIncubator'
    house = pd.read_csv(path+"/Zip_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_mon.csv")

    #add zipcode lat/lon
    zipcode = pd.read_csv(path+"/us-zip-code-latitude-and-longitude.csv")
    
    ziplist = house['RegionName'].tolist()
    house_zip = [[0]*2]*house.shape[0]
    for iloc in range(zipcode.shape[0]):
        if zipcode['Zip'][iloc] in ziplist:
            currzip = zipcode['Zip'][iloc]
            zip_ind= house[ziplist == currzip].index.values
            house_zip[zip_ind[0]] = zipcode['Latitude'][iloc], zipcode['Longitude'][iloc]      
            
    house[['Lat', 'Lon']]= pd.DataFrame(house_zip)
    return(zipcode, house)   

(zipcode,house) = load_house_prices_data()

In [7]:
def conv_ind_datetime():
    house_better = house.T

    date_list = ['']*house_better.shape[0]
    for idate in range(house_better.shape[0]):
         if "-" in house_better.index[idate]: #if it's a date
                currdate = datetime.strptime(str(house_better.index[idate]), '%Y-%m-%d').date()
                date_list[idate] = currdate
    house_better['Date'] = pd.to_datetime(date_list)
    return house_better

house_better = conv_ind_datetime()

In [ ]:
def match_T_to_house():
    #T_Pr has fewer locations, so use that just to get the unique lat and lon
    T_P_latlons =T_Pr_dates.set_index(['Date']).groupby(['Lat','Lon']).size().reset_index().rename(columns = {0: "num meas"})
    
    temppd_house = pd.DataFrame(np.zeros((house_better.shape[0],0)))
    temppd_house['Date'] = pd.to_datetime(house_better['Date'].tolist(), format='%Y-%m-%d %f') 
    
    latstouse = house_better.loc['Lat'][0:30343].astype(str).astype(float)
    lonstouse = house_better.loc['Lon'][0:30343].astype(str).astype(float)
   
    final = pd.DataFrame(np.zeros((0, temppd_house.shape[1]+T_Pr_dates.shape[1])), columns = {'Date','Lat','Lon','Price','day','index','level_0', 'month',
             'prcp','tmax','tmin','year'})
    
    for imeas in range(T_P_latlons.shape[0]):
        start_time = time.time()
        lat = T_P_latlons['Lat'].loc[imeas]
        lon = T_P_latlons['Lon'].loc[imeas]  
        
        if imeas>=1:
            prevlat = T_P_latlons['Lat'].loc[imeas-1]
            prevlon = T_P_latlons['Lon'].loc[imeas-1]
            if abs(prevlat - lat)<1E-4 and abs(prevlon - lon)<1E-4:
                continue
                
        cr1 = T_Pr_dates['Lat']== float(lat)
        cr2 = T_Pr_dates['Lon']== float(lon)
        temppd_TP = T_Pr_dates[cr1 & cr2].reset_index()
        temppd_TP['Date'] = pd.to_datetime(temppd_TP['Date'].astype('str'), format='%Y-%m-%d')
            
        for ispot in range(house_better.shape[1]-1):
            currlat = latstouse[ispot] #float(house_better.loc['Lat'][ispot])
            if abs(currlat - lat)<0.03:
                #print(ispot)
                currlon = float(lonstouse[ispot]) #house_better.loc['Lon'][ispot])
                if abs(currlon - lon)<0.03:
                #    print(currlat,lat, currlon,lon, imeas, ispot)
                    temppd_house[1] = house_better[ispot]            

                    merged = pd.merge(left=temppd_house, left_on='Date', right=temppd_TP, right_on='Date')
                    final = pd.concat([final, merged], axis=0) 
                #    print("run time for row %s is %s seconds" % (imeas, time.time() - start_time))
                    break          
        
    return final
final = match_T_to_house()

In [9]:
f, ax = plt.subplots()
plt.scatter(house_better.loc['Lon'][0:30342], house_better.loc['Lat'][0:30342])
plt.scatter(T_Pr_dates['Lon'],T_Pr_dates['Lat'])
plt.ylim([24,50]); plt.xlim([-130, -60]);
plt.show()

In [ ]:
#f, ax = plt.subplots()
#plt.scatter(house_better.loc['Lon'][0:20000], house_better.loc['Lat'][0:20000])
# plt.scatter(T_P_latlons['Lon'][0:10],T_P_latlons['Lat'][0:10])
# plt.ylim([24,50]); plt.xlim([-130, -60]);
# plt.show()

In [ ]:
final_group = final.groupby(['Lat','Lon'])
avg = final_group.mean()
std = final_group.std()
avg

plt.scatter(avg['prcp'],avg['Unnamed: 0'],  vmin=20, vmax=60, c= avg['tmin'])
plt.grid(axis='both', alpha=0.5)
plt.xlabel('precipitation (in)'); plt.ylabel('Housing Price ($)')
cbar = plt.colorbar()
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Tmax (F)', rotation=270)
plt.title('Housing Prices as a function of Local Precipitation')
plt.show()

In [ ]:
avg

At this stage, I have loaded the precipitation data, and I have loaded the housing data. I plan to use latitude and longitude to connect the housing data to the precipitation data (by city), so I have connected the precipitation data to latitudes and longitudes, as well as averaged the precipitation data by year and month, as the housing data is monthly. 
Due to time constraints, I have been unable to finish connecting the two different datasets.